In [37]:
#Plan is to make frame for all the advanced stats and all the basic stats, then merge them.

import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
import flask
from flask import request
import urllib.request, urllib.error
import requests
import response
import warnings
warnings.filterwarnings('ignore')



In [38]:
#IN THIS NOTEBOOK, WE WILL BE MAKING A DATASET OF MATCH STATS, MATCH BETTING LINES, TEAM ATTRIBUTES, AND TEAM EXPERIENCE
#EACH ROW WILL CONSIST OF THE DATA FOR ONE MATCH

#THE FIRST STEP IS TO GET THE COLUMNS NAMES FOR EACH OF THE BASIC STATISTICS

#THIS IS NOT THE FORMAT WE WANT THE TABLE IN. EACH OF THOSE STATISTIC LABELS SHOULD BE A COLUMN (FEATURE),
#SO THEY CAN BE USED TO PREDICT FUTURE OUTCOMES

#WE DON'T REALLY CARE ABOUT THE ACTUAL STATISTICS RIGHT NOW, WE JUST NEED TO GET THE LABELS 

btest=pd.read_html('https://www.footywire.com/afl/footy/ft_match_statistics?mid=9721')[17]
btest


,0,1,2,3,4,5,6
0,Head to Head,Head to Head,Head to Head,Head to Head,Head to Head,NaN,NaN
1,NaN,Carlton,Statistic,Richmond,NaN,NaN,NaN
2,NaN,238,Kicks,218,NaN,NaN,NaN
3,NaN,165,Handballs,140,NaN,NaN,NaN
4,NaN,403,Disposals,358,NaN,NaN,NaN
5,NaN,1.44,Kick to Handball Ratio,1.56,NaN,NaN,NaN
6,NaN,99,Marks,70,NaN,NaN,NaN
7,NaN,63,Tackles,62,NaN,NaN,NaN
8,NaN,30,Hitouts,40,NaN,NaN,NaN
9,NaN,23,Frees For,22,NaN,NaN,NaN


In [39]:
#THE "b" IN btest and aus_b STANDS FOR BASIC BECAUSE WE'RE EXTRACTING FROM THE BASIC STATS.
#THERE IS AN ALSO AN ADVANCED STATS PAGE WHICH WE'LL SCRAPE FROM IN THE NEXT CELL VIA THE SAME PROCESS.


btest.drop([0,4,5,6],axis=1,inplace=True)

btest.dropna(inplace=True)
btest.drop(0,inplace=True)

bheader=btest.iloc[0]
btest.columns=bheader

#ABOVE THE NULLS ARE CLEANED OUT
#BELOW WE TAKE EACH OF THE STATISTIC LABELS

cols=btest['Statistic'].to_list()
home_stats=btest.iloc[:,0].to_list()
away_stats=btest.iloc[:,2].to_list()

h_labels=[]
for i in cols:
    i=i+str('_home')
    h_labels.append(i)
    
a_labels=[]
for i in cols:
    i=i+str('_away')
    a_labels.append(i)

h_labels.extend(a_labels)
home_stats.extend(away_stats)

aus_b=pd.DataFrame()
aus_b=aus_b.append(pd.Series(home_stats),ignore_index=True)
aus_b.columns=(h_labels)
aus_b.rename(columns={'Statistic_home':'Home','Statistic_away':'Away'},inplace=True)
aus_b.columns

#EACH COLUMN OF THIS DATAFRAME IS A STAT FOR THE HOME TEAM AND AWAY TEAM
#AGAIN, IGNORE THE NUMBERS, WE ARE JUST USING THIS TO GET THE COLUMN NAMES
#WE WILL LOOP THROUGH THE PAGES AND SCRAPE ALL THE STATS LATER.

Index(['Home', 'Kicks_home', 'Handballs_home', 'Disposals_home',
       'Kick to Handball Ratio_home', 'Marks_home', 'Tackles_home',
       'Hitouts_home', 'Frees For_home', 'Frees Against_home',
       'Goals Kicked_home', 'Goal Assists_home', 'Behinds Kicked_home',
       'Rushed Behinds_home', 'Scoring Shots_home', 'Conversion_home',
       'Disposals Per Goal_home', 'Disps Per Scoring Shot_home',
       'Clearances_home', 'Clangers_home', 'Rebound 50s_home',
       'Inside 50s_home', 'In50s Per Scoring Shot_home',
       'Inside 50s Per Goal_home', '% In50s Score_home', '% In50s Goal_home',
       'Away', 'Kicks_away', 'Handballs_away', 'Disposals_away',
       'Kick to Handball Ratio_away', 'Marks_away', 'Tackles_away',
       'Hitouts_away', 'Frees For_away', 'Frees Against_away',
       'Goals Kicked_away', 'Goal Assists_away', 'Behinds Kicked_away',
       'Rushed Behinds_away', 'Scoring Shots_away', 'Conversion_away',
       'Disposals Per Goal_away', 'Disps Per Scoring Shot_a

In [40]:
#WE REPEAT THIS PROCESS FOR THE ADVANCED STATISTICS PAGE OF EACH MATCH (NOTICE THE DIFFERENT URLs)
atest=pd.read_html('https://www.footywire.com/afl/footy/ft_match_statistics?mid=9721&advv=Y#hd')[17]

atest.drop([0,4,5,6],axis=1,inplace=True)

atest.dropna(inplace=True)
atest.drop(0,inplace=True)

header=atest.iloc[0]
#atest=atest[1:]
atest.columns=header

cols=atest['Statistic'].to_list()
home_stats=atest.iloc[:,0].to_list()
away_stats=atest.iloc[:,2].to_list()

h_labels=[]
for i in cols:
    i=i+str('_home')
    h_labels.append(i)
    
a_labels=[]
for i in cols:
    i=i+str('_away')
    a_labels.append(i)

h_labels.extend(a_labels)
home_stats.extend(away_stats)

aus_a=pd.DataFrame()
aus_a=aus_a.append(pd.Series(home_stats),ignore_index=True)
aus_a.columns=(h_labels)
aus_a.rename(columns={'Statistic_home':'Home','Statistic_away':'Away'},inplace=True)
aus_a.columns

Index(['Home', 'Contested Possessions_home', 'Uncontested Possessions_home',
       'Effective Disposals_home', 'Disposal Efficiency %_home',
       'Clangers_home', 'Contested Marks_home', 'Marks Inside 50_home',
       'Clearances_home', 'Rebound 50s_home', 'One Percenters_home',
       'Bounces_home', 'Goal Assists_home', '% Goals Assisted_home',
       'Centre Clearances_home', 'Stoppage Clearances_home',
       'Score Involvements_home', 'Metres Gained_home', 'Turnovers_home',
       'Intercepts_home', 'Tackles Inside 50_home', 'Away',
       'Contested Possessions_away', 'Uncontested Possessions_away',
       'Effective Disposals_away', 'Disposal Efficiency %_away',
       'Clangers_away', 'Contested Marks_away', 'Marks Inside 50_away',
       'Clearances_away', 'Rebound 50s_away', 'One Percenters_away',
       'Bounces_away', 'Goal Assists_away', '% Goals Assisted_away',
       'Centre Clearances_away', 'Stoppage Clearances_away',
       'Score Involvements_away', 'Metres Gained

In [41]:
#WE PERFORM THE SAME PROCESS OF CLEANING THE TABLE AND MAKING COLUMNS FOR EACH TEAMS ATTRIBUTES/EXPERIENCE


atts=pd.read_html('https://www.footywire.com/afl/footy/ft_match_statistics?mid=9721')[18]
atts=atts[[1,2,3]]
atts=atts[atts[2].isnull()==False]
atts.reset_index(inplace=True,drop=True)
atts=atts.loc[0:11,:]
atts.drop([0,1,6,7],inplace=True)
attributes=atts[2].to_list()
home_atts=atts.iloc[:,0].to_list()
away_atts=atts.iloc[:,2].to_list()

h_alabels=[]
for i in attributes:
    i=i+str('_home')
    h_alabels.append(i)
    
a_alabels=[]
for i in attributes:
    i=i+str('_away')
    a_alabels.append(i)

h_alabels.extend(a_alabels)
home_atts.extend(away_atts)

att_df=pd.DataFrame()
att_df=att_df.append(pd.Series(home_atts),ignore_index=True)
att_df.columns=(h_alabels)
att_df.columns

Index(['Height_home', 'Weight_home', 'Age_home', 'Games_home',
       'Less than 50_home', '50 to 99_home', '100 to 149_home',
       '150 or more_home', 'Height_away', 'Weight_away', 'Age_away',
       'Games_away', 'Less than 50_away', '50 to 99_away', '100 to 149_away',
       '150 or more_away'],
      dtype='object')

In [42]:
#NEXT WE ARE GOING TO LOOP THROUGH EACH MATCH FOR THE PAST 5 YEARS AND SCRAPE ALL THE DATA
#THIS WILL TAKE A WHILE
#YOU CAN COMMENT OUT THE 9000 NUMBERS AND NOT USE ALL THE 6000s JUST TO TEST IT OUT


#(YOU CAN GO BACK FURTHER BUT THE ADVANCED STATS PAGE DOESNT HAVE AS MANY STATISTICS)


In [43]:
#THESE 4 DIGIT NUMBERS IN THE URL IDENTIFY THE MATCH. AT SOME POINT IN THE 5 YEARS, THE WEBSITE WENT FROM USING
#NUMBERS IN 6000's to 9000's

#ADDITIONALLY, EVERY ONCE IN A WHILE, A NUMBER IS SKIPPED, SO WE IMPLEMENT A "try". 
#IF WE TRY A NUMBER THAT DOESNT BELONG TO A URL, IT WILL RESULT IN AN ERROR.
#HOWEVER, WITH THE "try" and "pass" WE TELL THE SCRIPT TO SKIP OVER THIS ERROR AND CONTINUE COLLECTING DATA 

url_list=[]
for num_set in range(5964,6370):
    url_list.append(num_set)
for num_set2 in range(9298,9928):
    url_list.append(num_set2)
    

#OPENING LISTS TO STORE THE HOME AND AWAY GAME SPREADS AND MONEYLINES    
home_lines=[]
away_lines=[]
home_money=[]
away_money=[]

au_adv=pd.DataFrame()
au_basic=pd.DataFrame()
attribute_df=pd.DataFrame()
 
    
#THE COUNTER WILL LET US KNOW WHEN EACH TIME WE COLLECT THE DATA FOR 207 MATCHES (THE NUMBER OF MATCHES IN AN AFL SEASON)    
counter=0
for url in url_list:
    try:
        #THIS IS SIMILAR TO HOW WE ACQUIRED THE COLUMN NAMES ABOVE EXCEPT NOW WE'RE FOCUSED ON THE ACTUAL STATISTICS
        adv=pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_statistics?mid={url}&advv=Y#hd')[17]
        
        adv.drop([0,4,5,6],axis=1,inplace=True)

        adv.dropna(inplace=True)
        adv.drop(0,inplace=True)

        adv_header=adv.iloc[0]
        adv.columns=adv_header

        cols=adv['Statistic'].to_list()
        home_adv=adv.iloc[:,0].to_list()
        away_adv=adv.iloc[:,2].to_list()

        home_adv.extend(away_adv)


        au_adv=au_adv.append(pd.Series(home_adv),ignore_index=True)


        bas=pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_statistics?mid={url}')[17]

        bas.drop([0,4,5,6],axis=1,inplace=True)

        bas.dropna(inplace=True)
        bas.drop(0,inplace=True)

        bas_header=bas.iloc[0]
        bas.columns=bas_header

        cols=bas['Statistic'].to_list()
        home_basic=bas.iloc[:,0].to_list()
        away_basic=bas.iloc[:,2].to_list()

        home_basic.extend(away_basic)


        au_basic=au_basic.append(pd.Series(home_basic),ignore_index=True)



        atts=pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_statistics?mid={url}')[18]
        
        atts=atts[[1,2,3]]
        atts=atts[atts[2].isnull()==False]
        atts.reset_index(inplace=True,drop=True)
        atts=atts.loc[0:11,:]
        atts.drop([0,1,6,7],inplace=True)
        
        home_atts=atts.iloc[:,0].to_list()
        away_atts=atts.iloc[:,2].to_list()
        home_atts.extend(away_atts)
        attribute_df=attribute_df.append(pd.Series(home_atts),ignore_index=True)



        lines=pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_statistics?mid={url}&advv=Y#hd')[6]
        
        lines=lines.loc[[3,4]]
        home_info=lines.loc[3]
        away_info=lines.loc[4]

        for i in home_info.str.split(' '):
            home_lines.append(i[-3])
            home_money.append(i[-5])

        for i in away_info.str.split(' '):
            away_lines.append(i[-3])
            away_money.append(i[-5])

        counter+=1
        if  counter==207:
            print('207 games complete')
            counter=0     
            
    except:
        pass





#WE HAVE SCRAPED ADVANCED STATS, BASIC STATS, ATTRIBUTES, AND BETTING INFO.
#THEY ARE IN THEIR OWN SEPARATE DATAFRAMES (EXCEPT BETTING INFO WHICH IS STORED IN LISTS),
#SO EVENTUALLY WE WILL HAVE TO COMIBNE THEM TO MAKE ONE DATASET


207 games complete
207 games complete
207 games complete
207 games complete


In [44]:

#PUTTING THE COLUMN NAMES ON THE DATAFRAMES
au_adv.columns=aus_a.columns
au_basic.columns=aus_b.columns
attribute_df.columns=att_df.columns

#ADDING THE BETTING INFO FOR EACH MATCH INTO THE ADVANCED DATAFRAME
au_adv['home_line']=home_lines
au_adv['away_line']=away_lines
au_adv['home_money']=home_money
au_adv['away_money']=away_money

#CLEANING THE BETTING DATA
i=0
for i in range(i,len(au_adv)):
    au_adv.loc[i,'home_money']=au_adv.loc[i,'home_money'].replace(',','')
    au_adv.loc[i,'away_money']=au_adv.loc[i,'away_money'].replace(',','')
    au_adv.loc[i,'away_line']=au_adv.loc[i,'away_line'].replace('+','')
    au_adv.loc[i,'home_line']=au_adv.loc[i,'home_line'].replace('+','')
    
#JOINING THE ADVANCED STATS DATAFRAME AND THE ATTRIBUTES DATAFRAME
au=pd.concat([au_adv,attribute_df],axis=1)

#JOINING THE ADVANCED STATS DATAFRAME AND THE BASIC STATS DATAFRAME
au=pd.concat([au,au_basic],axis=1)

#GETTING RID OF DUPLICATE COLUMN NAMES
au=au.loc[:,~au.columns.duplicated()]

In [45]:
#NOW WE'RE GOING TO DO SOME WORK ON THE FEATURES


#DROP UNNECESSARY FEATURES
#SOME OF THESE FEATURES ARE COMBINED TO MAKE OTHERS OR ARE VERY SIMILAR TO OTHERS

#YOU DON'T HAVE TO GET RID OF THESE, IT WAS JUST MY CHOICE

au.drop(['Centre Clearances_home','Centre Clearances_away','Stoppage Clearances_home','Stoppage Clearances_away',
        'Turnovers_home','Turnovers_away','Disposals_home','Disposals_away','Kicks_home','Kicks_away',
        'Handballs_home','Handballs_away','Hitouts_home','Hitouts_away','Goal Assists_home','Goal Assists_away',
        'Scoring Shots_home','Scoring Shots_away','% In50s Score_home','% In50s Score_away','% In50s Goal_home',
         '% In50s Goal_away'],axis=1,inplace=True)

#CHANGING FROM STRING TO INTEGER
au['Goals Kicked_home']=au['Goals Kicked_home'].astype(int)
au['Behinds Kicked_home']=au['Behinds Kicked_home'].astype(int)
au['Rushed Behinds_home']=au['Rushed Behinds_home'].astype(int)

au['Goals Kicked_away']=au['Goals Kicked_away'].astype(int)
au['Behinds Kicked_away']=au['Behinds Kicked_away'].astype(int)
au['Rushed Behinds_away']=au['Rushed Behinds_away'].astype(int)


#CREATING SCORE FEATURE BY ADDING UP THE GOALS AND BEHINDS
au['Score_home']=au['Goals Kicked_home']*6+au['Behinds Kicked_home']+au['Rushed Behinds_home']
au['Score_away']=au['Goals Kicked_away']*6+au['Behinds Kicked_away']+au['Rushed Behinds_away']

#MAKING BEHINDS FEATURE BY ADDING RUSHED BEHINDS AND KICKED BEHINDS
au['Behinds_home']=au['Behinds Kicked_home']+au['Rushed Behinds_home']
au['Behinds_away']=au['Behinds Kicked_away']+au['Rushed Behinds_away']

#DROPPING THE SEPERATE TYPES OF BEHINDS
au.drop(['Behinds Kicked_home','Behinds Kicked_away','Rushed Behinds_home','Rushed Behinds_away'],axis=1,inplace=True)


#CLEANING THE HEIGHT, WEIGHT, AND AGE SO WE CAN ULTIMATELY MAKE THEM THE CORRECT DATA-TYPE

hwa=au[['Height_home','Height_away','Weight_home','Weight_away','Age_home','Age_away']]

height_list=[]
for i in hwa['Height_home']:
    i=i[:-2]
    height_list.append(i)
hwa['Height_home']=height_list

aheight_list=[]
for i in hwa['Height_away']:
    i=i[:-2]
    aheight_list.append(i)
hwa['Height_away']=aheight_list



weight_list=[]
for i in hwa['Weight_home']:
    i=i[:-2]
    weight_list.append(i)
hwa['Weight_home']=weight_list

aweight_list=[]
for i in hwa['Weight_away']:
    i=i[:-2]
    aweight_list.append(i)
hwa['Weight_away']=aweight_list




Age_year=[]
Age_month=[]
for i in hwa['Age_home'].str.split(' '):
    Age_year.append(i[0])
    Age_month.append(i[1])


aAge_year=[]
aAge_month=[]
for i in hwa['Age_away'].str.split(' '):
    aAge_year.append(i[0])
    aAge_month.append(i[1])


hwa['hage_year']=Age_year
hwa['aage_year']=aAge_year
hwa['hage_month']=Age_month
hwa['aage_month']=aAge_month


Age_listm=[]
for i in hwa['hage_month']:
    i=i[:-3]
    Age_listm.append(i)
hwa['hage_month']=Age_listm

aAge_listm=[]
for i in hwa['aage_month']:
    i=i[:-3]
    aAge_listm.append(i)
hwa['aage_month']=aAge_listm

Age_listy=[]
for i in hwa['hage_year']:
    i=i[:-2]
    Age_listy.append(i)
hwa['hage_year']=Age_listy

aAge_listy=[]
for i in hwa['aage_year']:
    i=i[:-2]
    aAge_listy.append(i)
hwa['aage_year']=aAge_listy

hwa['hage_year']=hwa['hage_year'].astype(int)
hwa['aage_year']=hwa['aage_year'].astype(int)
hwa['hage_month']=hwa['hage_month'].astype(int)
hwa['aage_month']=hwa['aage_month'].astype(int)

#I MEASURED THE AVERAGE AGE OF THE TEAM IN MONTHS, YOU CAN DO YEARS OR WHATEVER YOU FIND TO BE BEST

hwa['Age_home']=hwa['hage_year']*12+hwa['hage_month']
hwa['Age_away']=hwa['aage_year']*12+hwa['aage_month']
hwa.drop(['hage_year','hage_month','aage_year','aage_month'],axis=1,inplace=True)

au['Height_home']=hwa['Height_home']
au['Height_away']=hwa['Height_away']
au['Weight_home']=hwa['Weight_home']
au['Weight_away']=hwa['Weight_away']
au['Age_home']=hwa['Age_home']
au['Age_away']=hwa['Age_away']

#CREATING EXPERIENCE POINTS OUT OF THE DIFFERENT RANGE OF TOTAL GAMES PLAYED BY THE PLAYERS ON EACH TEAM
exps=au[['Less than 50_home','50 to 99_home', '100 to 149_home', '150 or more_home',
        'Less than 50_away','50 to 99_away', '100 to 149_away', '150 or more_away']]
exps=exps.astype(int)
au['Exp_home']=exps['150 or more_home']*4+exps['100 to 149_home']*3+exps['50 to 99_home']*2+exps['Less than 50_home']
au['Exp_away']=exps['150 or more_away']*4+exps['100 to 149_away']*3+exps['50 to 99_away']*2+exps['Less than 50_away']
au.drop(['Less than 50_home','50 to 99_home', '100 to 149_home', '150 or more_home',
        'Less than 50_away','50 to 99_away', '100 to 149_away', '150 or more_away'],axis=1,inplace=True)

In [46]:
#NOW WERE MAKING A SEPARATE TABLE FOR THE PERCENTAGE FEATURES
#CLEAN THEM UP TO EVENTUALLY CHANGE THE DATA-TYPE
#ADD THE CLEANED DATA BACK TO THE MAIN DATAFRAME

percents=au[['Conversion_home','Conversion_away','Disposal Efficiency %_home','Disposal Efficiency %_away','% Goals Assisted_home','% Goals Assisted_away']]
percents=percents.apply(lambda x: x.str.replace('%','') if x.dtype=="object" else x)

au['Conversion_home']=percents['Conversion_home']
au['Conversion_away']=percents['Conversion_away']
au['Disposal Efficiency %_home']=percents['Disposal Efficiency %_home']
au['Disposal Efficiency %_away']=percents['Disposal Efficiency %_away']
au['% Goals Assisted_home']=percents['% Goals Assisted_home']
au['% Goals Assisted_away']=percents['% Goals Assisted_away']

In [47]:
#FINALLY WE CHANGE ALL OUR NUMERICAL DATA TO FLOAT TYPE
#TO DO THE WHOLE DATAFRAME AT ONCE, I REMOVED HOME AND AWAY (THE ONLY NON-NUMERICAL DATA IN THE FRAME)
#AFTER CHANGING THE DATA-DTYPE, I SIMPLY ADDED THE HOME AND AWAY FEATURES BACK IN.

home_away=au[['Home','Away']]
au.drop(['Home','Away'],axis=1,inplace=True)
au=au.astype(float)
au['Home']=home_away['Home']
au['Away']=home_away['Away']

au.to_csv('au_cleaning.csv')

#ADDED THE SEASON FEATURE T0 THE CSV ITSELF

In [67]:
#NOW WE NEED TO GET GENERAL MATCH DATA (DATE,TIME,VENUE,SEASON,WEEK,ETC.)
#SIMILAR TO HOW WE ACQUIRED THE STATS...
#FOR EACH SEASON WE LOOP THROUGH, SCRAPE, AND CLEAN THE MATCH INFORMATION

matches=pd.DataFrame()
for seas in range(2015,2020):
    fixtures=pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_list?year={seas}#finals')[8]
    fixtures[1]=fixtures[1].str.replace(' v ',':')
    fixtures.drop([3,5,6],axis=1,inplace=True)
    fixtures.columns=['Date','Teams','Venue','Result']
    fixtures=fixtures[fixtures['Date']!='Date']
    fixtures['Round']=np.NaN
    fixtures['Round']=fixtures['Date'][(fixtures['Date'].str.contains('Round'))|(fixtures['Date'].str.contains('Final'))]
    fixtures['Round'].fillna(method='ffill',inplace=True)
    fixtures=fixtures[fixtures['Venue']!='BYE']
    fixtures.dropna(inplace=True)
    fixtures=fixtures[fixtures['Round']!=fixtures['Teams']]

    home_list=[]
    away_list=[]
    home_score=[]
    away_score=[]
    for i in fixtures['Teams'].str.split(':'):
        home_list.append(i[0])
        away_list.append(i[1])

    for i in fixtures['Result'].str.split('-'):
        home_score.append(i[0])
        away_score.append(i[1])

    fixtures['Home']=home_list
    fixtures['Away']=away_list
    fixtures['Score_home']=home_score
    fixtures['Score_away']=away_score
    fixtures['season']=seas

    fixtures.drop(['Teams','Result'],axis=1,inplace=True)
    fixtures.reset_index(drop=True,inplace=True)

    #SETTING UP LISTS TO STORE THE TIME, DAY OF WEEK, AND MONTH/DATE
    
    time=[]
    dow=[]
    md=[]

    for i in fixtures['Date'].str.split(' '):
        dow.append(i[0])
        time.append(i[3])
        md.append(i[1]+i[2])

    #CLEANING DATE,TIME, DAY OF WEEK, AND MONTH/DATE
    
    fixtures['Time']=time
    fixtures['Weekday']=dow
    fixtures['Date']=md
    fixtures['Date']=fixtures['Date']+fixtures['season'].astype(str)

    fixtures['Date']=fixtures['Date'].astype(str)
    fixtures['Month']=np.NaN
    fixtures['dd']=np.NaN
    i=0
    for i in range(i,len(fixtures)):
        fixtures.loc[i,'Month']=fixtures.loc[i,'Date'][-7:-4]
        fixtures.loc[i,'dd']=fixtures.loc[i,'Date'][:-7]

    fixtures['Date']=fixtures['season'].astype(str)+'-'+fixtures['Month']+'-'+fixtures['dd']
    fixtures.drop('dd',axis=1,inplace=True)
    
    
    #FIXING THE ROUND(MATCHWEEK) FEATURE
    #FIRST, MAKE THE PLAYOFF ROUNDS THE SAME FORMAT AS THE REGULAR SEASON ROUNDS
    
    fixtures['Round'][fixtures['Round']=='Qualifying Final']='Round 24'
    fixtures['Round'][fixtures['Round']=='Elimination Final']='Round 24'
    fixtures['Round'][fixtures['Round']=='Semi Final']='Round 25'
    fixtures['Round'][fixtures['Round']=='Preliminary Final']='Round 26'
    fixtures['Round'][fixtures['Round']=='Grand Final']='Round 27'

    #NEXT, GET RID OF "Round" IN EACH POINT AND STRIP OUT WHITE SPACE

    i=0
    for i in range(i,len(fixtures)):
        fixtures.loc[i,'Round']=fixtures.loc[i,'Round'][6:]
        fixtures.loc[i,'Round']=fixtures.loc[i,'Round'].strip()
    
    #MAKE DATE, ROUND, SCORE FEATURES THE PROPER DATA-TYPE
    fixtures['Date']=pd.to_datetime(fixtures['Date'])
    fixtures['Round']=fixtures['Round'].astype(int)
    fixtures['Score_home']=fixtures['Score_home'].astype(int)
    fixtures['Score_away']=fixtures['Score_away'].astype(int)
    
    
    
    
    #NOW WE'RE GOING TO CREATE A REST FEATURE BY GETTING THE DIFFERENCE IN DATE FROM THE LAST GAME TO THE CURRENT GAME FOR EACH TEAM

    days_since_home=fixtures.sort_values(['season','Round','Date'])[['season','Round','Date','Home']]
    days_since_away=fixtures.sort_values(['season','Round','Date'])[['season','Round','Date','Away']]
    days_since=days_since_home.append(days_since_away,ignore_index=True)

    days_since['Home'].fillna(days_since['Away'],inplace=True)
    days_since.sort_values(['season','Round','Date'],inplace=True)
    days_since=days_since[['season','Round','Date','Home']]
    days_since.rename(columns={'Home':'team'},inplace=True)
    
    #CREATE A DICTIONARY 
    team_dict = {}
    for key in days_since['team'].unique():                     #FOR EVERY TEAM...
        team_dict[key] = days_since[days_since['team'] == key].reset_index(drop=True)  # THE KEY OF THE DICTIONARY IS THE TEAM AND THE VALUES ARE THE FEATURES FOR THAT TEAM IN THE DAYS_SINCE DATAFRAME
    

    days = pd.DataFrame()

    #NOW WE WANT TO CREATE ANOTHER COLUMN WITH THE DATES FOR EACH TEAMS GAMES SHIFTED 1 ROW DOWN

    for yr in days_since['season'].unique():        #FOR EACH SEASON
        for tm in days_since['team'].unique():               #FOR EACH TEAM
            data = team_dict[tm].copy()                   #COPY OF THE TEAM_DICT CREATED ABOVE
            data = data[data['season'] == yr]  #SET VARIABLE "data" EQUAL TO THE DICTIONARY WITH 1 OF THE SEASON'S INFO FOR 1 OF THE TEAMS
                                                            
            data.loc[:, 'shifted'] = data['Date'].shift()     #CREATE A NEW COLUMN THAT SHIFTS EACH GAME 1 WEEK FORWARD
                                                                                            
            days = days.append(data)                    #PROCESS IS LOOPED THROUGH EACH TEAM IN A SEASON THEN MOVES ONTO THE NEXT SEASON

    days['rest']=days['Date'].subtract(days['shifted']) #GET THE DIFFERENCE OF DATES FOR DAYS REST
    
    
    
    #MERGE THE DAYS DATAFRAME AND FIXTURES DATAFRAME BACK TOGETHER AND GET RID OF THE DUPLICATE COLUMNS
    fixtures=fixtures.merge(days.drop('shifted',axis=1),left_on=['season','Round','Date','Home'],
               right_on=['season','Round','Date','team'],how='left')
    fixtures.rename(columns={'rest':'home_rest'},inplace=True)

    fixtures=fixtures.merge(days.drop('shifted',axis=1),left_on=['season','Round','Date','Away'],
               right_on=['season','Round','Date','team'],how='left')
    fixtures.rename(columns={'rest':'away_rest'},inplace=True)

    fixtures=fixtures.drop_duplicates(subset=['season','Round','Date','Home','Away'])

    fixtures.drop(['team_x','team_y'],axis=1,inplace=True)

    #MAKE PLAYOFF FEATURE (1 IF IT IS A PLAYOFF GAME, 0 IF NOT)
    fixtures['Playoff']=np.NaN
    fixtures['Playoff'][fixtures['Round']>=24]=1
    fixtures['Playoff'].fillna(0,inplace=True)

    #MAKE AND EAST TO WEST TRAVEL AND WEST TO EAST TRAVEL FEATURE
    
    #A TEAM MIGHT NOT PLAY TO ITS FULL CAPABILITY AFTER A LONG FLIGHT, SO WE WANT TO INCORPORATE THIS INTO PREDICTIONS
    
    fixtures['etw']=np.NaN
    fixtures['etw'][(fixtures['Home']=='Fremantle')&(fixtures['Away']!='West Coast')&(fixtures['Round']!=27)]=1
    fixtures['etw'][(fixtures['Home']=='West Coast')&(fixtures['Away']!='Fremantle')&(fixtures['Round']!=27)]=1
    fixtures['etw'].fillna(0,inplace=True)
    fixtures['wte']=np.NaN
    fixtures['wte'][(fixtures['Away']=='Fremantle')&(fixtures['Home']!='West Coast')&(fixtures['Round']!=27)]=1
    fixtures['wte'][(fixtures['Away']=='West Coast')&(fixtures['Home']!='Fremantle')&(fixtures['Round']!=27)]=1
    fixtures['wte'].fillna(0,inplace=True)
    
    #CREATE FEATURES THAT DESCRIBE THE RESULT OF THE MATCH
    fixtures['Home_win']=np.NaN
    fixtures['Home_win'][fixtures['Score_home']>fixtures['Score_away']]=1
    fixtures['Home_win'].fillna(0,inplace=True)
    
    fixtures['Away_win']=np.NaN
    fixtures['Away_win'][fixtures['Score_away']>fixtures['Score_home']]=1
    fixtures['Away_win'].fillna(0,inplace=True)
    
    fixtures['Draw']=np.NaN
    fixtures['Draw'][fixtures['Score_away']==fixtures['Score_home']]=1
    fixtures['Draw'].fillna(0,inplace=True)
    
    #APPEND THE FIXTURES DATAFRAME TO THE END OF THE EMPTY MATCHES FRAME CREATED BEFORE WE ENTERED THIS LOOP
    #THIS WHOLE PROCESS WILL BE LOOPED THROUGH FOR EACH SEASON
    matches=matches.append(fixtures,ignore_index=True)
    print(f'matches appended {seas} data')

matches appended 2015 data
matches appended 2016 data
matches appended 2017 data
matches appended 2018 data
matches appended 2019 data


In [68]:
#FILL IN THE NULLS FOR ROUND 1 OF EACH SEASON WITH 100
matches['home_rest'].fillna(pd.Timedelta('100 days'),inplace=True)
matches['away_rest'].fillna(pd.Timedelta('100 days'),inplace=True)

matches.head(60)

,Date,Venue,Round,Home,Away,Score_home,Score_away,season,Time,Weekday,Month,home_rest,away_rest,Playoff,etw,wte,Home_win,Away_win,Draw
0,2015-04-02,MCG,1,Carlton,Richmond,78,105,2015,7:20pm,Thu,Apr,100 days,100 days,0.0,0.0,0.0,0.0,1.0,0.0
1,2015-04-04,MCG,1,Melbourne,Gold Coast,115,89,2015,1:40pm,Sat,Apr,100 days,100 days,0.0,0.0,0.0,1.0,0.0,0.0
2,2015-04-04,ANZ Stadium,1,Sydney,Essendon,72,60,2015,4:35pm,Sat,Apr,100 days,100 days,0.0,0.0,0.0,1.0,0.0,0.0
3,2015-04-04,Gabba,1,Brisbane,Collingwood,74,86,2015,6:20pm,Sat,Apr,100 days,100 days,0.0,0.0,0.0,0.0,1.0,0.0
4,2015-04-04,Marvel Stadium,1,Western Bulldogs,West Coast,97,87,2015,7:20pm,Sat,Apr,100 days,100 days,0.0,0.0,1.0,1.0,0.0,0.0
5,2015-04-05,Marvel Stadium,1,St Kilda,GWS,78,87,2015,1:10pm,Sun,Apr,100 days,100 days,0.0,0.0,0.0,0.0,1.0,0.0
6,2015-04-05,Adelaide Oval,1,Adelaide,North Melbourne,140,63,2015,2:50pm,Sun,Apr,100 days,100 days,0.0,0.0,0.0,1.0,0.0,0.0
7,2015-04-05,Domain Stadium,1,Fremantle,Port Adelaide,75,68,2015,4:40pm,Sun,Apr,100 days,100 days,0.0,1.0,0.0,1.0,0.0,0.0
8,2015-04-06,MCG,1,Hawthorn,Geelong,123,61,2015,3:20pm,Mon,Apr,100 days,100 days,0.0,0.0,0.0,1.0,0.0,0.0
9,2015-04-10,Domain Stadium,2,West Coast,Carlton,131,62,2015,6:10pm,Fri,Apr,6 days,8 days,0.0,1.0,0.0,1.0,0.0,0.0


In [69]:

afl_csv=pd.read_csv('au_cleaning.csv')
afl_csv['Season']=matches['season']

#MERGE THE MEXTURES DATAFRAME WITH THE MATCH STATS DATAFRAME
afl=matches.merge(afl_csv,left_on=['season','Home','Away','Score_home','Score_away'],
                 right_on=['Season','Home','Away','Score_home','Score_away'],how='left')

afl.drop(['Unnamed: 0','Season'],axis=1,inplace=True)
#FROM HERE YOU CAN DO MORE OF YOUR OWN FEATURE ENGINEERING...
#YOU CAN CREATE ROLLING AVERAGE FOR WHICHEVER STATISTICS YOU LIKE TO USE THEM TO PREDICT
    #- THE PROCESS OF DOING THIS IS SIMILAR TO HOW WE MADE THE DAYS REST FEATURE.

In [70]:
afl.head(60)

,Date,Venue,Round,Home,Away,Score_home,Score_away,season,Time,Weekday,...,Conversion_away,Disposals Per Goal_away,Disps Per Scoring Shot_away,Inside 50s_away,In50s Per Scoring Shot_away,Inside 50s Per Goal_away,Behinds_home,Behinds_away,Exp_home,Exp_away
0,2015-04-02,MCG,1,Carlton,Richmond,78,105,2015,7:20pm,Thu,...,50.0,23.67,11.83,54.0,1.80,3.60,12.0,15.0,51.0,53.0
1,2015-04-04,MCG,1,Melbourne,Gold Coast,115,89,2015,1:40pm,Sat,...,54.2,23.15,12.54,47.0,1.96,3.62,13.0,11.0,44.0,44.0
2,2015-04-04,ANZ Stadium,1,Sydney,Essendon,72,60,2015,4:35pm,Sat,...,60.0,37.67,22.60,44.0,2.93,4.89,12.0,6.0,55.0,54.0
3,2015-04-04,Gabba,1,Brisbane,Collingwood,74,86,2015,6:20pm,Sat,...,46.2,28.83,13.31,60.0,2.31,5.00,8.0,14.0,45.0,45.0
4,2015-04-04,Marvel Stadium,1,Western Bulldogs,West Coast,97,87,2015,7:20pm,Sat,...,82.4,27.14,22.35,40.0,2.35,2.86,13.0,3.0,42.0,51.0
5,2015-04-05,Marvel Stadium,1,St Kilda,GWS,78,87,2015,1:10pm,Sun,...,44.4,29.42,13.07,52.0,1.93,4.33,12.0,15.0,44.0,44.0
6,2015-04-05,Adelaide Oval,1,Adelaide,North Melbourne,140,63,2015,2:50pm,Sun,...,50.0,35.11,17.56,47.0,2.61,5.22,14.0,9.0,45.0,62.0
7,2015-04-05,Domain Stadium,1,Fremantle,Port Adelaide,75,68,2015,4:40pm,Sun,...,55.6,39.00,21.67,42.0,2.33,4.20,9.0,8.0,61.0,57.0
8,2015-04-06,MCG,1,Hawthorn,Geelong,123,61,2015,3:20pm,Mon,...,38.1,43.12,16.43,42.0,2.00,5.25,21.0,13.0,58.0,56.0
9,2015-04-10,Domain Stadium,2,West Coast,Carlton,131,62,2015,6:10pm,Fri,...,52.9,35.56,18.82,35.0,2.06,3.89,11.0,8.0,49.0,49.0


In [71]:
afl.columns

Index(['Date', 'Venue', 'Round', 'Home', 'Away', 'Score_home', 'Score_away',
       'season', 'Time', 'Weekday', 'Month', 'home_rest', 'away_rest',
       'Playoff', 'etw', 'wte', 'Home_win', 'Away_win', 'Draw',
       'Contested Possessions_home', 'Uncontested Possessions_home',
       'Effective Disposals_home', 'Disposal Efficiency %_home',
       'Clangers_home', 'Contested Marks_home', 'Marks Inside 50_home',
       'Clearances_home', 'Rebound 50s_home', 'One Percenters_home',
       'Bounces_home', '% Goals Assisted_home', 'Score Involvements_home',
       'Metres Gained_home', 'Intercepts_home', 'Tackles Inside 50_home',
       'Contested Possessions_away', 'Uncontested Possessions_away',
       'Effective Disposals_away', 'Disposal Efficiency %_away',
       'Clangers_away', 'Contested Marks_away', 'Marks Inside 50_away',
       'Clearances_away', 'Rebound 50s_away', 'One Percenters_away',
       'Bounces_away', '% Goals Assisted_away', 'Score Involvements_away',
       'Metres

In [150]:
afl.to_csv('aussie_rules_football.csv')